In [1]:
!pip install networkx

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep
from networkx import Graph

op = webdriver.ChromeOptions()
op.add_argument('--headless')
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s, options=op)
G = Graph()

url_oslo = 'https://twitter.com/hashtag/Oslo?src=hashtag_click'
url_bergen = 'https://twitter.com/hashtag/Bergen?src=hashtag_click'

[WDM] - Downloading: 100%|██████████| 6.83M/6.83M [00:00<00:00, 11.7MB/s]


In [3]:
def crawl_hash(graph:Graph, url:str, min_len_crawl:int):
    """takes some url and searches for hastags to then crawl through the found hastags, 
    it visits min_len_crawl number of hashtags in addition to the original url

    Args:
        graph (Graph): networkx.Graph type object
        url (str): start url
        min_len_crawl (int): how many hashtags to visit.
    """

    hashtags = []
    visited = []
    while len(visited) < min_len_crawl:
        driver.get(url)
        sleep(3)

        try:
            elements = driver.find_elements(By.XPATH, "//a[contains(@href, '/hashtag')]")
        except:
            continue
        
        for element in elements:
            # extracting the hasthag link for each hashtag and removing any video or image versions of a hastag
            hashtag = element.get_attribute('href').split('&')[0]
            
            if hashtag not in hashtags:
                hashtags.append(hashtag)

            if url != hashtag:
                graph.add_edge('#'+url[28:-18], '#'+hashtag[28:-18])
        
        # make a list of hastags and always visit the oldest hastag
        url = hashtags[0]
        visited.append(hashtags.pop(0))

In [4]:
crawl_hash(G, url_oslo, 20)
crawl_hash(G, url_bergen, 20)

IndexError: list index out of range

In [ ]:
from networkx import write_graphml
write_graphml(G, 'OBHashtags.graphml')